#### This notebook contains routines for constructing a database from raw data downloaded from cathdb.info/download and CATH-API



In [1]:
from tst.utils_db import *


### Specify the version tag of your session...
### Create a new tag if not existing
v_curr = verify_version('putative')

print(v_curr)



putative


In [27]:
#### Debug failed attempts of domain_stat_fill()

# # dname = flst[0]
# dname = '3rfwA01'
# d = domain.objects.get(domain_id = dname)
# try:
#     domain_stat_fill(d,env = env) 
# except Exception as e:
#     flst.append(d.domain_id)
#     print("Unable to process %s for %s" % (d.domain_id, str(e)))

In [ ]:
### Attempt to calculate structure-based statistics for a collection of domains

#from tst.utils_db import *

dset = domain.objects.filter( classification__version = v_curr )
# dset = dset.annotate(version = 'classification__version')

#### Initialise env for modeller, and specify the path to search for pdbs
env =init_env()

lst = dset
imax = len(lst)
flst = []

for i,d in enumerate(lst):
    try:
        domain_stat_fill(d,env = env) 
    except Exception as e:
        flst.append(d.domain_id)
        print("Unable to process %s for %s" % (d.domain_id, str(e)))
    if not i % 100:
        print('%d of %d'%(i,imax))

In [2]:
#### Calculate superfamily-based statistics

from time import time
t0 = time()
from tst.utils_db import *

level_curr = level.objects.get(letter = 'H')
# node_set =  classification.objects.filter(version = v_curr)
node_set =  classification.objects.filter(version__id__lte = v_curr.id)



lst = node_set.filter(level = level_curr)

c = counter(lst, per = 100)

failcount = 0
for d in lst:
    c.count()
    with transaction.atomic():
#     if 1:
        failcount += homsf_stat_fill(d) == 0

print '%d instances failed' % failcount
print('Ended after %.4f' % (time()-t0))  # len(lst)
# homsf_stat_fill(lst[0])
# print(str(lst[0]))

0 of 6874
100 of 6874
200 of 6874
300 of 6874
400 of 6874
500 of 6874
600 of 6874
700 of 6874
800 of 6874
900 of 6874
1000 of 6874
1100 of 6874
1200 of 6874
1300 of 6874
1400 of 6874
1500 of 6874
1600 of 6874
1700 of 6874
1800 of 6874
1900 of 6874
2000 of 6874
2100 of 6874
2200 of 6874
2300 of 6874
2400 of 6874
2500 of 6874
2600 of 6874
2700 of 6874
2800 of 6874
2900 of 6874
3000 of 6874
3100 of 6874
3200 of 6874
3300 of 6874
3400 of 6874
3500 of 6874
3600 of 6874
3700 of 6874
3800 of 6874
3900 of 6874
4000 of 6874
4100 of 6874
4200 of 6874
4300 of 6874
4400 of 6874
4500 of 6874
4600 of 6874
4700 of 6874
4800 of 6874
4900 of 6874
5000 of 6874
5100 of 6874
5200 of 6874
5300 of 6874
5400 of 6874
5500 of 6874
5600 of 6874
5700 of 6874
5800 of 6874
5900 of 6874
6000 of 6874
6100 of 6874
6200 of 6874
6300 of 6874
6400 of 6874
6500 of 6874
6600 of 6874
6700 of 6874
6800 of 6874
0 instances failed
Ended after 216.3868


In [1]:
h = classification.objects.get_bytree('1.10.30.10')[0]
# print(h)
# classification.objects.get_bytree

1.10.30.10.0


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import numpy as np

data = np.random.randn(20, 40)

pipeline = Pipeline([
    ('pca', PCA(n_components=2)),
    ('scaling', StandardScaler()),
                    ])

pipeline.fit_transform(data)
data.shape

sset = h.classification_set.all()
(xs,ys,lbls) = zip(*sset.values_list('domain__domain_stat__atom_count','domain__domain_stat__nbpair_count','id'))
data = np.array([xs,ys])
# lbls
nd = pipeline.fit_transform(data.T)
xs = nd[:,0]
ys = nd[:,1]
# print nd.shape
# print data.shape
# scatterplot_dict(xs,ys,lbls, regress = 0, show = 1)

pip_pc_std = Pipeline([
    ('pca', PCA(n_components=2)),
    ('scaling', StandardScaler()),
                    ])
pcdata = pip_pc_std.fit_transform(data.T)
pcxs,pcys = pcdata.T
# pcy = pcdata[:,1]
print pcxs.shape
(pcxs**2+pcys**2) - np.array(sset.values_list('domain__domain_stat__maha_dist')).T
# def pc_std(data):

(235,)


In [7]:
from tst.utils import *

h = classification.objects.get_bytree('1.10.30.10')[0]
sset = h.classification_set

(xs,ys,lbls) = zip(*sset.values_list('domain__domain_stat__atom_count','domain__domain_stat__nbpair_count','id'))
# results = PCA(data)

In [2]:
from tst.utils_db import *
h = classification.objects.get_bytree("2.40.50.140")[0]
print homsf_stat_fill(h)

1


In [6]:
### plotting mahalanobis distance
# del scatterplot_dict.__module__
from tst.utils import *

h = classification.objects.get_bytree('1.10.30.10')[0]
sset = h.classification_set

(xs,ys,lbls) = zip(*sset.values_list('parent__node_stat__Rsq_NBcount_Acount','domain__domain_stat__maha_dist','id'))

# lsts = zip(*sset.values_list('parent__node_stat__Rsq_NBcount_Acount','domain__domain_stat__maha_dist','id'))
# scatterplot_dict(*lsts, regress=0, show = 1)
scatterplot_dict(xs,ys,lbls, regress = 0, show = 1)


/home/shouldsee/.local/lib/python2.7/site-packages/matplotlib/axes/_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "



Note: if you're in the IPython notebook, mpld3.show() is not the best command
      to use. Consider using mpld3.display(), or mpld3.enable_notebook().
      See more information at http://mpld3.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [01/Aug/2017 18:22:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2017 18:22:36] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2017 18:22:36] "GET /mpld3.js HTTP/1.1" 200 -



stopping Server...


{'axes': [{'axes': [{'fontsize': 12.0,
     'grid': {'gridOn': False},
     'nticks': 8,
     'position': 'bottom',
     'scale': u'linear',
     'tickformat': None,
     'tickvalues': None,
     'visible': True},
    {'fontsize': 12.0,
     'grid': {'gridOn': False},
     'nticks': 7,
     'position': 'left',
     'scale': u'linear',
     'tickformat': None,
     'tickvalues': None,
     'visible': True}],
   'axesbg': '#FFFFFF',
   'axesbgalpha': None,
   'bbox': (0.082438271604938274,
    0.097924382716049418,
    0.86978395061728397,
    0.85429783950617277),
   'collections': [{'alphas': [0.15],
     'edgecolors': ['#0000FF'],
     'edgewidths': (1.0,),
     'facecolors': ['#0000FF'],
     'id': 'el2728139946320366544',
     'offsetcoordinates': 'data',
     'offsets': 'data01',
     'pathcoordinates': 'display',
     'paths': [([[0.0, -0.5],
        [0.13260155, -0.5],
        [0.25978993539242673, -0.44731684579412084],
        [0.3535533905932738, -0.3535533905932738],
        

In [49]:
### plotting mahalanobis distance
# del scatterplot_dict.__module__
# from tst.utils import *
# from tst.views import *
from matplotlib import pyplot as plt
dom1="3.30.497.10"
dom2="2.30.39.10"

sf1 = classification.objects.get_bytree(dom1)[0]
sf2 = classification.objects.get_bytree(dom1)[0]
fields=[			'domain_stat__atom_count',
			'domain_stat__nbpair_count',
			'id',
]

fields = [
			'domain_stat__pcx',
			'domain_stat__pcy',
			'id',
			]
xss=[];yss=[];lblss=[];css=[]


# msg = '';
# plt.close()
# siz = [450,400]
# DPI=100.
# fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
# ax1 = fig.add_subplot(111)
# # points = ax1.scatter(xs,y2s, colors,

sset = homsf2domain(dom1)



(xs,ys,lbls)=zip(*sset.values_list(*fields))
                     
# xss+=xs
# yss+=ys
# lblss+=lbls
# css+=len(xs)*[0]
xss+=[xs]
yss+=[ys]
lblss+=[lbls]
css+=[len(xs)*[0]]

# ax1.scatter(xs,ys,color=1)

sset = homsf2domain(dom2)
                     
(xs2,ys2,lbls2)=zip(*sset.values_list(*fields))
# 
xss+=[xs2]
yss+=[ys2]
lblss+=[lbls2]
css+=[len(xs2)*[2]]

# ax1.scatter(xs,ys,color=2)

# lsts = zip(*sset.values_list('parent__node_stat__Rsq_NBcount_Acount','domain__domain_stat__maha_dist','id'))
# scatterplot_dict(*lsts, regress=0, show = 1)

# subplot_kwargs = {
# 			# 'xlim':[0,500],
# 			'xlim':[0,500E1],
# 			'ylim':[0,800E3],
# 			'xlabel':'atom_count',
# 			'ylabel':'Non-bonding pair count',
# 			},

subplot_kwargs={
			'xlim':[-5,10],
			'ylim':[-9,7],
			'xlabel': 'pcx (relative structure size)',
			'ylabel': 'pcy (packedness)',
# 			"regress":False,
			}

scatterplot_dict(xs,ys,lbls,
                 xs2,ys2,lbls2,
                 regress = 0, show = 1   , 
                 **subplot_kwargs
                )



Note: if you're in the IPython notebook, mpld3.show() is not the best command
      to use. Consider using mpld3.display(), or mpld3.enable_notebook().
      See more information at http://mpld3.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [04/Aug/2017 14:54:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2017 14:54:28] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2017 14:54:28] "GET /mpld3.js HTTP/1.1" 200 -



stopping Server...


{'axes': [{'axes': [{'fontsize': 12.0,
     'grid': {'alpha': 1.0,
      'color': '#B0B0B0',
      'dasharray': 'none',
      'gridOn': True},
     'nticks': 7,
     'position': 'bottom',
     'scale': u'linear',
     'tickformat': None,
     'tickvalues': None,
     'visible': True},
    {'fontsize': 12.0,
     'grid': {'alpha': 1.0,
      'color': '#B0B0B0',
      'dasharray': 'none',
      'gridOn': True},
     'nticks': 10,
     'position': 'left',
     'scale': u'linear',
     'tickformat': None,
     'tickvalues': None,
     'visible': True}],
   'axesbg': '#FFFFFF',
   'axesbgalpha': None,
   'bbox': (0.16377700617283952,
    0.1779991319444445,
    0.76038966049382717,
    0.76825086805555542),
   'collections': [{'alphas': [0.35],
     'edgecolors': ['#FF0000'],
     'edgewidths': (1.0,),
     'facecolors': ['#FF0000'],
     'id': 'el22612140539325142096',
     'offsetcoordinates': 'data',
     'offsets': 'data01',
     'pathcoordinates': 'display',
     'paths': [([[0.0, -0.5

In [40]:
def scatterplot_dict(xs1,y2s1,ids1,xs2,y2s2,ids2, colors = None,lbls=None, forced_lbls=None, regress = True, show = False,**kwargs):   
#     if not lbls:
#         lbls = ids
    # if forced_lbls:
    #     lbls = forced_lbls
    #     print( "lbls settled")
    # print( "lbls not settled")

    # font = {'family' : 'normal',
    #     'weight' : 'bold',
    #     'size'   : 16}
    # # matplotlib.rc('font', **font)
    SMALL_SIZE = 12
    MEDIUM_SIZE = 16
    BIGGER_SIZE = 18
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    # plt.rc('axes', titleweight='bold')    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = 'Ubuntu'
    plt.rcParams['font.monospace'] = 'Ubuntu Mono'
    plt.rcParams['font.weight'] = 10
    # plt.rcParams['font.size'] = 10
    # plt.rcParams['axes.labelsize'] = 10
    plt.rcParams['text.usetex']=False
    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['axes.titleweight'] = 'bold'


    idx = [not (x is None or y is None) for x,y in zip(xs2,y2s2)]
    xs = [x for x,y in zip(xs2,idx) if y]
    y2s = [x for x,y in zip(y2s2,idx) if y]
    ids = [x for x,y in zip(ids2,idx) if y]
    
#     lbls=ids2
#     xs = np.array(xs);
#     y2s = np.array(y2s)
    # 
    # xs = xs.flat[idx]
    # print(len(idx))
    # y2s = y2s.flat[idx]
    # ids = [x for x,y in zip(ids,idx) if y]
    # lbls = lbls[idx]
    
    msg = '';
    plt.close()
    siz = [450,400]
    DPI=100.
    fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
    ax1 = fig.add_subplot(111,**kwargs)
    points = ax1.scatter(xs,y2s, 
        # colors,
#         edgecolor = 'b',facecolor = 'b',
        edgecolor = 'r',facecolor = 'r',
        marker ='o',
        c = colors,
         s =100,
         alpha = 0.35,
         # **kwargs
         )
    # if 1:
    
    
    idx = [not (x is None or y is None) for x,y in zip(xs1,y2s1)]
    xs = [x for x,y in zip(xs1,idx) if y]
    y2s = [x for x,y in zip(y2s1,idx) if y]
    ids = [x for x,y in zip(ids1,idx) if y]
#     lbls=ids1
    xs = np.array(xs);
    y2s = np.array(y2s)
    # 
    # xs = xs.flat[idx]
    # print(len(idx))
    # y2s = y2s.flat[idx]
    # ids = [x for x,y in zip(ids,idx) if y]
    # lbls = lbls[idx]
    
#     msg = '';
#     plt.close()
#     siz = [450,400]
#     DPI=100.
#     fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
#     ax1 = fig.add_subplot(111,**kwargs)
    points = ax1.scatter(xs,y2s, 
        # colors,
        edgecolor = 'b',facecolor = 'b',
        # edgecolor = 'r',facecolor = 'r',
        marker ='o',
        c = colors,
         s =100,
         alpha = 0.35,
         # **kwargs
         )
    # if 1:
    
    
    title = ''
    if regress:
        ((m,b),C) = np.polyfit(xs, y2s, 1, cov =True) ### This covariance seems different from the one generated by np.cov()
        # (m,b) = np.polyfit(xs, y2s, 1, cov = False)
        C = np.cov(xs,y2s)
        r_sq = cov2corr(C).flat[1] ** 2
  
        res2  = y2s - ( m*xs + b)
        outs = MAD_outlier(res2,3.0)
        outs = (outs * (res2<0));
        xso = xs[outs]
        y2so = y2s[outs]
        sc1 = ax1.scatter(xso,y2so,s=33,
            color='r',
            marker = 'x',
            label='outlier num = %d'%sum(outs))

        title += '''
        y  = %5.3f * x + %5.3f , 
        R_squared=%3.3f '''% (m,b,r_sq)
        xmm = np.array([min(xs),max(xs)])
        ax1.plot(xmm, m*xmm + b, 'g--',label = 'linear regression fit')

    oxlim = ax1.get_xlim()
    oylim = ax1.get_ylim()
    ax1.axis('auto')
    axlim = ax1.get_xlim()
    aylim = ax1.get_ylim()
    ax1.set_xlim(oxlim)
    ax1.set_ylim(oylim)

    ax1.legend()
    
    # ax1.legend(bbox_to_anchor=(1.1, 1.05))
    ax1.set_title(
        title)

    ax1.grid(True, which='both')
    plt.tight_layout()
 
    tooltip = mpld3.plugins.PointLabelTooltip(points, labels=lbls)
    mpld3.plugins.connect(fig, tooltip)
    mpld3.plugins.connect(fig, HelloWorld(points,lbls,ids))
    mpld3.plugins.connect(fig, RectPlugin(xlim=axlim,ylim=aylim))
    jdict = mpld3.fig_to_dict(fig);
    jstr = json.dumps(jdict);

    if show:
        mpld3.show()
    else:
        plt.close(fig)
        # fig.show()
        # plt.close("all")
    return(jdict)
### Benchmarking object retrival in tree-structured database
### Compared naive search ".get()"  against ".get_bytree()"
### Consider using Django-mptt instead of writing custom functions

# node = '1.10.40.30.0'
# node = '1.10.60.10.3'
# levels=['Class',
# 'arch',
# 'topo',
# 'homsf',
# 's35',
# 's60',
# 's95',
# 's100'];

node = '2.40.500.20.0'
qdict = dict(zip(levels,node.split('.')))
c0 = None
# c0 = classification.objects.get(**dict(zip(levels,node.split('.')[:2]+['0'])))
from time import time

n = 2
t0=time()
for i in range(n):
    try:
        c1 = classification.objects.get(**qdict)
    except:
        c1 = [];
# print
t1=time()
for i in range(n):
    c2 = classification.objects.get_bytree(node,qnode = c0)[0]
t2=time()    
print('start at',c0)
print(c1)
print(t1-t0)
print(c2)
print(t2-t1)


('start at', None)
[]
0.000247001647949
2.40.500.20.0
0.0172209739685


In [5]:
#### Fetch data from cathdb.info

import urllib2

from StringIO import StringIO
import gzip


putative_s35_url = 'http://download.cathdb.info/cath/releases/daily-release/newest/cath-b-s35-newest.gz'

request = urllib2.Request( putative_s35_url )
# request.add_header('Accept-encoding', 'gzip')
response = urllib2.urlopen(request)
# if response.info().get('Content-Encoding') == 'gz':
if 1:
    buf = StringIO(response.read())
    f = gzip.GzipFile(fileobj=buf)
    data = f.read()# len(f)
# type(f)
# help(f)
# lines = data.splitlines()

In [5]:
node = '2.40.500.0'
qdict = dict(zip(levels,node.split('.')))
c1 = classification.objects.filter(**qdict)
for c in c1:
# c1
#     print(c)
    if c.classification_set.all().exists():
        print(c.classification_set.all())
    else:
        c.delete()
        print('no domain')

<QuerySet [<classification: 2.40.500.10.0>]>


In [2]:
# c2.classification_set.all()
levels

[None, 'root', 'Class', 'arch', 'topo', 'homsf', 's35', 's60', 's95', 's100']

In [4]:
# c = c2.classification_set.all()[0]

from copy import copy
# c=copy(c2)
# c.pk = None
# c.level = None
ndict = c2.node_dict()
lv = c2.level.id
c = classification.objects.create(level_id=lv+1,version_id = 2,**ndict)
# c.level = level.objects.get(id = c2.level_id+1);
c.save()
print(c2.level_id,c.level_id)
print(c2.level,c.level)

c.delete()
# c2.level_id

NameError: name 'c2' is not defined

In [10]:
from domutil.util import *
from copy import copy
def verify_version(ver):
    #### Check whether this version is recorded in 'version' table
#     try:
    vset = version.objects.filter(name=ver)
    if vset.count() > 1:
        raise Exception,'multiple version with name %s'%(ver)
    elif not vset.exists():
        v = version(name=ver)
        v.save()
    else:
        v = vset[0]
    return v

verify_version('putative')

def parse_domain(line):
    global cc
    lst = line.split()
    
    domain_id = lst[0]
    ver = 'putative'
#     ver = lst[1]
#     node_str = lst[2]
    homsf_str = lst[1]    
    chopping = lst[2]
    
    ### verify version
    v = verify_version(ver)

    #### Check whether this node exists in 'classification' table
#     (node,success) = classification.objects.get_bytree(node_str)
    (node,success) = classification.objects.get_bytree(homsf_str)
    
    ### recursively  create new superfamily if not existing
#     print(levels)
    if not success: 
        lv = node.level.id
        lst = [int(x) for x in homsf_str.split('.')]
        lst = [0,0] + lst
        while lv != len(lst)-1:
            
            lv += 1
            
            ndict = node.node_dict()
            ndict[levels[lv]] = lst[lv]
            
            node = classification.objects.create(level_id = lv,
                                                 version = v,
                                                 parent = node,
                                                 **ndict)
            node.save()
            
#             print('created %s for %s'%(str(node),homsf_str))
            cc+=1
            
    ####
    conflict = 0
    try:
        d = node.domain
        check = 1
    except:
        d = domain.objects.create(classification = node,
                                 domain_id = domain_id)
        d.save()
        check = 0

    if check:
        if str(node) in mapdict.keys():
            pass
        elif d.domain_id != domain_id:
            conflict = 1
            
            
            node = copy(node);
            node.pk = None

            lv = node.level.id
            setattr(node, levels[lv],
                    max( node.parent.classification_set.values_list( levels[lv],flat = True ) )+1
                   )
            node.version = v
            node.save()

            d = domain.objects.create(classification = node,
                                 domain_id = domain_id)
            d.save()

        else:
            pass

    return([node,conflict])
    
        
        
    #### if superfamily exists, check whether domain_id agreed
    conflict = 0;
#     print(homsf,homsf_str,success)#     print(line)

    if success:
        pass

    #### Otherwise, write conflict to file
        

In [1]:
import urllib2


from StringIO import StringIO
import gzip


putative_s35_url = 'http://download.cathdb.info/cath/releases/daily-release/newest/cath-b-s35-newest.gz'

request = urllib2.Request( putative_s35_url )
# request.add_header('Accept-encoding', 'gzip')
response = urllib2.urlopen(request)
# if response.info().get('Content-Encoding') == 'gz':
if 1:
    buf = StringIO(response.read())
    f = gzip.GzipFile(fileobj=buf)
    data = f.read()# len(f)
# type(f)
# help(f)
# lines = data.splitlines()
mapdict = {}

In [2]:
dlist = domain.objects.values_list('domain_id',flat = True)
s = set(dlist)
dlist = list(s)
len(dlist)

34106

[]


In [4]:
dlist = domain.objects.values_list('domain_id',flat = True)
s = set(dlist)
dlist = list(s)

lines = data.splitlines()
from domutil.util import *
newlines = [];
c = counter(lines, per = 1000)

for line in lines:
    c.count()
    if line.split()[0] not in dlist:
        newlines.append(line)
# print(newlines)
    

0 of 31964
1000 of 31964
2000 of 31964
3000 of 31964
4000 of 31964
5000 of 31964
6000 of 31964
7000 of 31964
8000 of 31964
9000 of 31964
10000 of 31964
11000 of 31964
12000 of 31964
13000 of 31964
14000 of 31964
15000 of 31964
16000 of 31964
17000 of 31964
18000 of 31964
19000 of 31964
20000 of 31964
21000 of 31964
22000 of 31964
23000 of 31964
24000 of 31964
25000 of 31964
26000 of 31964
27000 of 31964
28000 of 31964
29000 of 31964
30000 of 31964
31000 of 31964
['4g3oA00 1.10.8.10.3 446-498:A', '4heoA00 1.10.8.10.4 2-56:A', '4wp2F00 1.10.8.10.5 600-655:F', '4f2lA00 1.10.8.10.7 284-344:A', '1zv1A00 1.10.8.10.8 6-64:A', '2qhoD00 1.10.8.10.9 178-226:D', '3bbzA00 1.10.8.10.16 344-391:A', '3t6pA02 1.10.8.10.24 386-451:A', '3h4jA03 1.10.8.10.29 299-354:A', '4dbgB02 1.10.8.10.33 529-589:B', '1ixrB03 1.10.8.10.34 139-191:B', '4dbgB03 1.10.8.10.39 590-627:B', '5ji2E02 1.10.8.10.42 110-243:E', '2lvnC00 1.10.8.10.48 453-504:C', '1r4gA00 1.10.8.10.50 516-568:A', '2knaA01 1.10.8.10.51 22-75:A', '1

In [8]:
data = '\n'.join(newlines)
with open('tmp','w') as f:
    f.write(data)
# print(len(newlines))

In [11]:
cc=0
lines = data.splitlines()
c = counter(lines, per = 1000)
for line in lines:
    n,conf = parse_domain(line)
    if conf:
#         print(n.domain.domain_id,line)
        mapdict[ line.split()[1] + '.0' ] = str(n)
#         maplst +=[ (line.split()[1],str(n).rstrip('.0')) ]
#         print(line.split()[1],str(n).rstrip('.0'))
        
    else:
        pass
#         print('success',n.domain.domain_id,line)
    c.count()
#     pass
#     print(line)
    pass
print(cc)

0 of 12951
1000 of 12951
2000 of 12951
3000 of 12951
4000 of 12951
5000 of 12951
6000 of 12951
7000 of 12951
8000 of 12951
9000 of 12951
10000 of 12951
11000 of 12951
12000 of 12951
6054


In [6]:
dupes = ( domain.objects.values('domain_id')
    .annotate(Count('id')) 
#     .order_by()
     .filter(id__count__gt=1)
)
# dupd.count()
# dupd[0]

dupset = domain.objects.filter(domain_id__in=[item['domain_id'] for item in dupes])
print len(dupes)

10243


In [6]:
d = dupset[0]
doms = d.classification.parent.classification_set.all().values_list('domain')
doms[0]

(265326L,)

In [11]:
# dupes[0]
from domutil.util import *

ct = counter(dupes)
# for d in dupset:
#     doms = d.classification.parent.classification_set.all().domain

cid = [];
did = [];
cdel = classification.objects.none()
ddel = domain.objects.none()


for ele in dupes:
# domain_id = dupes[0]['domain_id']
    domain_id = ele['domain_id']
    doms = dupset.filter(domain_id=domain_id)
    vlst = doms.values_list('classification__version__id')
    tlst = [x == min(vlst) for x in vlst]
    for t,dom in zip(tlst,doms) :
        ct.count()
        if not t:
#             print(dom.domain_id)
            
            c = dom.classification
            cid += [c.id]
            did += [dom.id]
#             cdel = cdel | c
#             ddel = ddel | dom
#             c.delete()
#             dom.delete()
# print(vlst)
# min(doms)
# print(dupd)

0 of 10243
100 of 10243
200 of 10243


KeyboardInterrupt: 

In [21]:
dset = domain.objects.all()
# dset[0].__dict__
dset.count()

53760

In [17]:
# dset = domain.objects.filter(classification__version__name='putative')
# dset.count()

26524

In [8]:
homsf_str='1.10.8.370.1'
# node = classification.objects.get_bytree(homsf_str)
(node,success) = classification.objects.get_bytree(homsf_str)
node

<classification: 1.10.8.0>

In [13]:
# line = lines[0]
# print(line)

# print(line.split())

qset = classification.objects.filter(version_id=version.objects.get(name='putative'),level__letter = 'S')
qset.count()
qset.delete()
# parse_domain(line)
# print(n.version)

(46382L,
 {u'tst.classification': 23191L,
  u'tst.domain': 23191L,
  u'tst.domain_stat': 0L,
  u'tst.node_stat': 0L})

In [ ]:
ver

In [1]:
# domain.objects.filter(classification__version__name='putative')
# classification.objects.filter(version__id__gte=2)
vset = version.objects.filter(id__gte=3)
print(vset)
vset.delete()
# raise Exception
# for v in vset:
#     v.delete()
#     print('del')
# classification

<QuerySet [<version: putative>]>


(570L,
 {u'tst.classification': 286L,
  u'tst.domain': 283L,
  u'tst.domain_stat': 0L,
  u'tst.node_stat': 0L,
  u'tst.version': 1L})

In [20]:
print('hi')

hi


In [6]:
def cathb2clf(data):
    lines = data.splitlines()
    newlines = []
    for l in lines:
#         l = '    '+ l
        lst = l.split()

        domain_id = lst[0]
        ver = 'putative'
    #     ver = lst[1]
    #     node_str = lst[2]
        node_str = lst[1]    
#         chopping = lst[2]

        nodelst = node_str.split('.') 
        nodelst += ( 9 - len(nodelst)) * ['1']
        l = [lst[0]] + nodelst + ['0','0'] 
        newlines += ['     '.join(l)]
    return('\n'.join(newlines))

# from django.conf import settings as djangoSettings
# print(djangoSettings.STATIC_ROOT)
fname = '../doc/cath-b-s35-newest.clfd'

with open(fname,'w') as f:
    d = cathb2clf(data)
    f.write(d)
# print(cathb2clf(data)[:100])
#     with open(fname) as f:
#         f.read

NameError: name 'homsf_stat_fill' is not defined

In [3]:
# (djangoSettings.__dict__)
# STATIC_ROOT
django.contrib.staticfiles

NameError: name 'django' is not defined

In [46]:
# import numpy as np 
# from domutil.util import *


# def homsf_stat_fill(h):
# # if 1:
#     if h.level.letter == 'H':
#         pass
#     else:
#         print "Node %s is not filled because it is not a homsf, but a '%s'" % (str(h),h.level.letter)
    
    
#     try:
#         nstat = h.node_stat
#     except:
#         nstat = node_stat(node = h)
#         nstat.save()
        

#     hset  = h.classification_set;
#     if hset.count()>10:
#         hset = hset.annotate(Acount=Avg("domain__domain_stat__atom_count"))
#         hset = hset.annotate(Rcount=Avg("domain__domain_stat__res_count"))
#         hset = hset.annotate(NBcount=Avg("domain__domain_stat__nbpair_count"))

#     #     hset.domain
#         l = hset.values_list('Acount','Rcount','NBcount')
#         a = np.array(l)
#     #     C = np.cov(a[:,0],a[:,1],a[:,2])
#         c = np.cov(a.T)
#         C = cov2corr(c); ## utils.cov2corr
# #         print(C.shape)
#         nstat.Rsq_NBcount_Acount = C[0,2] ** 2
#         nstat.Rsq_NBcount_Rcount = C[1,2] ** 2
#     else:
#         nstat.Rsq_NBcount_Acount = None
#         nstat.Rsq_NBcount_Rcount = None
#     try:
#         nstat.save()
#     except:
#         print nstat.__dict__
#     return h

# #     print l[:10]
# #     print l[1]
# #     print len(l)
# h = lst[2]
# homsf_stat_fill(h)